## DB에서 상장 기업 정보를 받아 네이버 증권 주가 수집하기

In [18]:
import os
import pandas as pd
from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pymysql
import time
import requests
from bs4 import BeautifulSoup as bs

In [3]:
pymysql.install_as_MySQLdb()
load_dotenv(dotenv_path = '.env_db')

True

In [5]:
# mysql에서 테이블 불러오기
engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('id')}:{os.getenv('pw')}@{os.getenv('host')}/{os.getenv('database')}")
conn = engine.connect()
data = pd.read_sql(f'2024_07_29_stock_company_info2', con=conn)
conn.close()

In [6]:
data

,증권종류,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,유가증권,산일전기,062040,"전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업","유입, 몰드, 주상, 건식 변압기 등",2024-07-29,12월,박동석,http://www.sanil.co.kr,경기도
1,유가증권,에이치에스효성,487570,기타 금융업,지주사업,2024-07-29,12월,조현상..,http://www.hshyosung.com,서울특별시
2,"코스닥, 투자주의종목",엔에이치스팩31호,481890,금융 지원 서비스업,금융지원서비스업,2024-07-26,12월,이시형,,서울특별시
3,코스닥,SK증권제13호스팩,473950,금융 지원 서비스업,기업인수목적 주식회사,2024-07-25,12월,임율표,,서울특별시
4,코스닥,엑셀세라퓨틱스,373110,기초 의약물질 제조업,CellCor SFD/CD(세포배양배지),2024-07-15,12월,이의일,,서울특별시
...,...,...,...,...,...,...,...,...,...,...
2700,"유가증권, KTOP30, KOSPI200, KRX300",유한양행,000100,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
2701,"유가증권, KOSPI200, KRX300",CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
2702,유가증권,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
2703,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


In [7]:
data['종목코드']

0       062040
1       487570
2       481890
3       473950
4       373110
         ...  
2700    000100
2701    000120
2702    000050
2703    000700
2704    003480
Name: 종목코드, Length: 2705, dtype: object

In [9]:
for code in data["종목코드"]:
    url = f"https://finance.naver.com/item/main.naver?code={code}"
    print(url)

https://finance.naver.com/item/main.naver?code=062040
https://finance.naver.com/item/main.naver?code=487570
https://finance.naver.com/item/main.naver?code=481890
https://finance.naver.com/item/main.naver?code=473950
https://finance.naver.com/item/main.naver?code=373110
https://finance.naver.com/item/main.naver?code=478110
https://finance.naver.com/item/main.naver?code=462870
https://finance.naver.com/item/main.naver?code=450330
https://finance.naver.com/item/main.naver?code=462350
https://finance.naver.com/item/main.naver?code=481850
https://finance.naver.com/item/main.naver?code=295310
https://finance.naver.com/item/main.naver?code=266350
https://finance.naver.com/item/main.naver?code=160190
https://finance.naver.com/item/main.naver?code=479880
https://finance.naver.com/item/main.naver?code=464080
https://finance.naver.com/item/main.naver?code=478440
https://finance.naver.com/item/main.naver?code=477340
https://finance.naver.com/item/main.naver?code=034230
https://finance.naver.com/it

In [23]:
url = f"https://finance.naver.com/item/main.naver?code=062040"
r = requests.get(url)
print(r.status_code)
print(r.text)
soup = bs(r.text, 'lxml')

200


	
	
	
	
<html lang='ko'>
<head>


	
		<title>산일전기 : 네이버페이 증권</title>
	
	




<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />

<meta http-equiv="Content-Script-Type" content="text/javascript">
<meta http-equiv="Content-Style-Type" content="text/css">
<meta name="apple-mobile-web-app-title" content="네이버페이 증권" />





	
    
        <meta property="og:url" content="https://finance.naver.com/item/main.naver?code=062040"/>
        
			
		    
		    	<meta property="og:title" content="산일전기 - 네이버페이 증권 : 네이버페이 증권"/>
		     
		
		
			
			   <meta property="og:description" content="관심종목의 실시간 주가를 가장 빠르게 확인하는 곳"/>
		    
		    
		
		 
			
			    <meta property="og:image" content="https://ssl.pstatic.net/static/m/stock/im/2016/08/og_stock-200.png"/>
		    
		    
		
    

<meta property="og:type" content="article"/>
<meta property="og:article:thumbnailUrl" content=""/>
<meta property="og:article:author" content="네이버페이 증권"/>
<meta property="og:article:author:url" content=

In [162]:
# 년월일
date = soup.select_one('em.date').text.split(' ')[0]
date

'2024.07.29'

In [164]:
# 이름
name = soup.select_one('h2').text
name

'산일전기'

In [165]:
# 종목코드
n_code = soup.select_one("span.code").text
n_code

'062040'

In [53]:
# 현재가
soup.select_one('em.no_up').text

'\n47,150\n47,150\n'

In [166]:
# 현재가
now = int(soup.select_one('p.no_today').text.split('\n')[2].replace(',', ''))
now

47150

In [73]:
# 변동 추출X
soup.select_one('p.no_exday').text.split('\n')

['',
 '전일대비',
 '',
 '상승',
 '12,150',
 '12,150',
 '',
 'l',
 '',
 '+',
 '34.71',
 '34.71',
 '%',
 '',
 '']

In [155]:
# 변동금액
change_num = int(soup.select_one('p.no_exday').text.split('\n')[4].replace(',', ''))
change_num

12150

In [153]:
# 변동룰
change = soup.select_one('p.no_exday').text.split('\n')[9:11]
change_rate = ''.join(change)
change_rate

'+34.71'

In [83]:
# 전일
soup.select_one('td.first > span.sptxt.sp_txt2').text

'전일'

In [167]:
# 전일
yester = int(soup.select_one('span.sptxt.sp_txt2 ~ em').text.split('\n')[1].replace(',', ''))
yester


35000

In [109]:
# 고가
soup.select_one('span.sptxt.sp_txt4 ~ em').text

'\n49,20049,200\n'

In [124]:
# 고가
high_num = int(soup.select_one('span.sptxt.sp_txt4 ~ em span.blind').text.replace(',', ''))
high_num

49200

In [144]:
# 거래량
vol = int(soup.select_one('span.sptxt.sp_txt9 + em span.blind').text.replace(',', ''))
vol

14961646

In [150]:
# 시가
pow = int(soup.select_one('span.sptxt.sp_txt3 ~ em span.blind').text.replace(',', ''))
pow

44900

In [151]:
# 저가
low = int(soup.select_one('span.sptxt.sp_txt5 ~ em span.blind').text.replace(',', ''))
low

40200

In [152]:
# 거래대금
transcation = int(soup.select_one('span.sptxt.sp_txt10 ~ em span.blind').text.replace(',', ''))
transcation

661929

In [ ]:
url = f"https://finance.naver.com/item/main.naver?code=062040"
r = requests.get(url)
print(r.status_code)
print(r.text)
soup = bs(r.text, 'lxml')

In [ ]:
pay_list_result = {}
for code in data["종목코드"]:
    url = f"https://finance.naver.com/item/main.naver?code={code}"
    #print(r.status_code)
    #print(r.text)
    soup = bs(r.text, 'lxml')
    #날짜
    date = soup.select_one('em.date').text.split(' ')[0]
    # 이름
    name = soup.select_one('h2').text
    # 종목코드
    n_code = soup.select_one("span.code").text
    now = int(soup.select_one('p.no_today').text.split('\n')[2].replace(',', ''))
    change_num = int(soup.select_one('p.no_exday').text.split('\n')[4].replace(',', ''))
    change = soup.select_one('p.no_exday').text.split('\n')[9:11]
    change_rate = ''.join(change)
    yester = int(soup.select_one('span.sptxt.sp_txt2 ~ em').text.split('\n')[1].replace(',', ''))
    high_num = int(soup.select_one('span.sptxt.sp_txt4 ~ em span.blind').text.replace(',', ''))
    vol = int(soup.select_one('span.sptxt.sp_txt9 + em span.blind').text.replace(',', ''))
    pow = int(soup.select_one('span.sptxt.sp_txt3 ~ em span.blind').text.replace(',', ''))
    low = int(soup.select_one('span.sptxt.sp_txt5 ~ em span.blind').text.replace(',', ''))
    transcation = int(soup.select_one('span.sptxt.sp_txt10 ~ em span.blind').text.replace(',', ''))

    #print(date, name, n_code, now, change_num, change_rate, yester, high_num, vol, pow, low, transcation)
    values = [date, name, n_code, now, change_num, change_rate, yester, high_num, vol, pow, low, transcation]
    keys = ['날짜', '이름', '종목코드', '현재가', '변동금액', '변동률', '전일', '고가', '거래량', '시가', '저가', '거래대금']

    for key,value in zip(keys, values):
        pay_list_result.setdefault(key, []).append(value)
        time.sleep(5)
        print('수집중')

final_result = pd.DataFrame(pay_list_result)
final_result





수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
수집중
